<a href="https://colab.research.google.com/github/harsh-agar/E-Net/blob/master/Frustum_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==1.13.1

In [2]:
!pip install hyperopt

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import os
os.chdir('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/tf_ops/sampling')
!bash tf_sampling_compile.sh

tf_sampling.cpp: In lambda function:
tf_sampling.cpp:20:40: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
     c->WithRank(c->input(0), 2, &dims1);
                                        ^
In file included from tf_sampling.cpp:8:0:
/usr/local/lib/python3.6/dist-packages/tensorflow/include/tensorflow/core/framework/shape_inference.h:394:10: note: declared here
   Status WithRank(ShapeHandle shape, int64 rank,
          ^~~~~~~~
tf_sampling.cpp:22:40: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
     c->WithRank(c->input(1), 2, &dim

In [5]:
import os
os.chdir('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master//pointnet2/tf_ops/grouping')
!bash tf_grouping_compile.sh

tf_grouping.cpp: In lambda function:
tf_grouping.cpp:22:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
         c->WithRank(c->input(1), 3, &dims2);
                                            ^
In file included from tf_grouping.cpp:8:0:
/usr/local/lib/python3.6/dist-packages/tensorflow/include/tensorflow/core/framework/shape_inference.h:394:10: note: declared here
   Status WithRank(ShapeHandle shape, int64 rank,
          ^~~~~~~~
tf_grouping.cpp: In lambda function:
tf_grouping.cpp:47:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-

In [6]:
import os
os.chdir('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master//pointnet2/tf_ops/3d_interpolation')
!bash tf_interpolate_compile.sh

tf_interpolate.cpp: In lambda function:
tf_interpolate.cpp:29:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
         c->WithRank(c->input(0), 3, &dims1);
                                            ^
In file included from tf_interpolate.cpp:8:0:
/usr/local/lib/python3.6/dist-packages/tensorflow/include/tensorflow/core/framework/shape_inference.h:394:10: note: declared here
   Status WithRank(ShapeHandle shape, int64 rank,
          ^~~~~~~~
tf_interpolate.cpp:31:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
         c->WithR

In [0]:
from __future__ import print_function

import sys
import os
import tensorflow as tf
import numpy as np

In [8]:
# os.path.dirname('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
BASE_DIR = os.path.dirname('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/')
ROOT_DIR = os.path.dirname('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/utils/')
sys.path.append(BASE_DIR)
sys.path.append(ROOT_DIR)
# sys.path.append(os.path.join(ROOT_DIR, 'utils'))

In [0]:
import tf_util
import numpy as np
import importlib

import csv
from pointnet_util import pointnet_sa_module, pointnet_sa_module_msg, pointnet_fp_module
# from model_util import VAl_CONSTANT, VAL_NUM_HEADING_BIN, VAL_NUM_SIZE_CLUSTER
# from model_util import NUM_HEADING_BIN, NUM_SIZE_CLUSTER, NUM_OBJECT_POINT
# from model_util import point_cloud_masking, get_center_regression_net
# from model_util import parse_output_to_tensors

In [0]:
# def get_instance_classification_v1_net(point_cloud, is_training, bn_decay):
#     l0_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3])
#     l0_points = tf.slice(point_cloud, [0,0,3], [-1,-1,1])

#     # Set abstraction layers
#     l1_xyz, l1_points = pointnet_sa_module_msg(l0_xyz, l0_points,
#         128, [0.2,0.4,0.8], [32,64,128],
#         [[32,32,64], [64,64,128], [64,96,128]],
#         is_training, bn_decay, scope='layer1')
#     l2_xyz, l2_points = pointnet_sa_module_msg(l1_xyz, l1_points,
#         32, [0.4,0.8,1.6], [64,64,128],
#         [[64,64,128], [128,128,256], [128,128,256]],
#         is_training, bn_decay, scope='layer2')
#     l3_xyz, l3_points, _ = pointnet_sa_module(l2_xyz, l2_points,
#         npoint=None, radius=None, nsample=None, mlp=[128,256,1024],
#         mlp2=None, group_all=True, is_training=is_training,
#         bn_decay=bn_decay, scope='layer3')
#     net = tf_util.fully_connected(tf.reshape(l3_points, [batch_size, 1024]), 512, scope='fc1-classification', bn=True, is_training=is_training, bn_decay=bn_decay)
#     net = tf_util.fully_connected(net, 256, scope='fc2-classification', bn=True,
#         is_training=is_training, bn_decay=bn_decay)
#     predicted_classes_scores = tf_util.fully_connected(net, NUM_CLASSES, activation_fn=tf.nn.softmax,
#         scope='fc3-classification')

#     l3_points = tf.concat([l3_points, tf.expand_dims(predicted_classes_scores, 1)], axis=2)
#     return l3_points, predicted_classes_scores

In [0]:
NUM_HEADING_BIN = 12
NUM_SIZE_CLUSTER = 8 # one cluster for each type
VAl_CONSTANT = 2 # 3 if 'x,y,z', 2 if 'x,y'
VAL_NUM_HEADING_BIN = 2
VAL_NUM_SIZE_CLUSTER = 3 # 4 if 'x,y,z', 3 if 'x,y'

# Set training configurations
BATCH_SIZE = 1
NUM_POINT = 1024
NUM_CLASSES = 2
NUM_CHANNEL = 4
NUM_OBJECT_POINT = 512
xyz_only = 1

batch_size = 1

In [0]:
def tf_gather_object_pc(point_cloud, mask, npoints=512):
	''' Gather object point clouds according to predicted masks.
	Input:
		point_cloud: TF tensor in shape (B,N,C)
		mask: TF tensor in shape (B,N) of 0 (not pick) or 1 (pick)
		npoints: int scalar, maximum number of points to keep (default: 512)
	Output:
		object_pc: TF tensor in shape (B,npoint,C)
		indices: TF int tensor in shape (B,npoint,2)
	'''
	def mask_to_indices(mask):
		indices = np.zeros((mask.shape[0], npoints, 2), dtype=np.int32)
		for i in range(mask.shape[0]):
			pos_indices = np.where(mask[i,:]>0.5)[0] #BxN
			# skip cases when pos_indices is empty
			if len(pos_indices) > 0: 
				if len(pos_indices) > npoints:
					choice = np.random.choice(len(pos_indices),
						npoints, replace=False)
				else:
					choice = np.random.choice(len(pos_indices),
						npoints-len(pos_indices), replace=True)
					choice = np.concatenate((np.arange(len(pos_indices)), choice))
				np.random.shuffle(choice)
				indices[i,:,1] = pos_indices[choice]
			indices[i,:,0] = i
		return indices

	indices = tf.py_func(mask_to_indices, [mask], tf.int32)  
	object_pc = tf.gather_nd(point_cloud, indices)
	return object_pc, indices

In [40]:
tf.reset_default_graph()
with tf.Graph().as_default():
#   with tf.device('/gpu:0'):
  is_training = tf.constant(False, dtype=tf.bool)
  bn_decay=None
  point_cloud = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_POINT, 6))
  batch_size = point_cloud.get_shape()[0].value
  num_point = point_cloud.get_shape()[1].value
  # Classification
  l0_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3])
  l0_points = tf.slice(point_cloud, [0,0,3], [-1,-1,1])

  # Set abstraction layers
  l1_xyz, l1_points = pointnet_sa_module_msg(l0_xyz, l0_points,
      128, [0.2,0.4,0.8], [32,64,128],
      [[32,32,64], [64,64,128], [64,96,128]],
      is_training, bn_decay, scope='layer1')
  l2_xyz, l2_points = pointnet_sa_module_msg(l1_xyz, l1_points,
      32, [0.4,0.8,1.6], [64,64,128],
      [[64,64,128], [128,128,256], [128,128,256]],
      is_training, bn_decay, scope='layer2')
  l3_xyz, l3_points, _ = pointnet_sa_module(l2_xyz, l2_points,
      npoint=None, radius=None, nsample=None, mlp=[128,256,1024],
      mlp2=None, group_all=True, is_training=is_training,
      bn_decay=bn_decay, scope='layer3')
  net = tf_util.fully_connected(tf.reshape(l3_points, [batch_size, 1024]), 512, scope='fc1-classification', bn=True, is_training=is_training, bn_decay=bn_decay)
  net = tf_util.fully_connected(net, 256, scope='fc2-classification', bn=True,
      is_training=is_training, bn_decay=bn_decay)
  predicted_classes_scores = tf_util.fully_connected(net, NUM_CLASSES, activation_fn=tf.nn.softmax,
      scope='fc3-classification')

  l3_points = tf.concat([l3_points, tf.expand_dims(predicted_classes_scores, 1)], axis=2)

  # Segmentation
  l2_points = pointnet_fp_module(l2_xyz, l3_xyz, l2_points, l3_points,
      [128,128], is_training, bn_decay, scope='fa_layer1')
  l1_points = pointnet_fp_module(l1_xyz, l2_xyz, l1_points, l2_points,
      [128,128], is_training, bn_decay, scope='fa_layer2')
  l0_points = pointnet_fp_module(l0_xyz, l1_xyz,
      tf.concat([l0_xyz,l0_points],axis=-1), l1_points,
      [128,128], is_training, bn_decay, scope='fa_layer3')

  # FC layers
  net = tf_util.conv1d(l0_points, 128, 1, padding='VALID', bn=True,
      is_training=is_training, scope='conv1d-fc1', bn_decay=bn_decay)
#     end_points['feats'] = net 
  net = tf_util.dropout(net, keep_prob=0.7,
      is_training=is_training, scope='dp1')
  logits = tf_util.conv1d(net, 2, 1,
      padding='VALID', activation_fn=None, scope='conv1d-fc2')

  # Masking
  mask = tf.slice(logits,[0,0,0],[-1,-1,1]) < \
    tf.slice(logits,[0,0,1],[-1,-1,1]) #True when object is present else false
  mask = tf.cast(mask, tf.float32) # BxNx1
  mask_count = tf.tile(tf.reduce_sum(mask,axis=1,keep_dims=True),
    [1,1,3]) # Bx1x3
  point_cloud_xyz = tf.slice(point_cloud, [0,0,0], [-1,-1,3]) # BxNx3
  mask_xyz_mean = tf.reduce_sum(tf.tile(mask, [1,1,3])*point_cloud_xyz,
    axis=1, keep_dims=True) # Bx1x3 ##Computes mean of all the mask points 'x,y,z' which have a object 'mask' present in it
  mask = tf.squeeze(mask, axis=[2]) # BxN
  mask_xyz_mean = mask_xyz_mean/tf.maximum(mask_count,1) # Bx1x3

  # Translate to masked points' centroid
  point_cloud_xyz_stage1 = point_cloud_xyz - \
    tf.tile(mask_xyz_mean, [1,num_point,1])
  if xyz_only: 
    point_cloud_stage1 = point_cloud_xyz_stage1
  else:
    point_cloud_features = tf.slice(point_cloud, [0,0,3], [-1,-1,-1])
    point_cloud_stage1 = tf.concat(\
      [point_cloud_xyz_stage1, point_cloud_features], axis=-1)
  num_channels = point_cloud_stage1.get_shape()[2].value
  object_point_cloud, _ = tf_gather_object_pc(point_cloud_stage1,
    mask, NUM_OBJECT_POINT)
  object_point_cloud.set_shape([batch_size, NUM_OBJECT_POINT, num_channels])
  #T-Net
  num_point = object_point_cloud.get_shape()[1].value
  net = tf.expand_dims(object_point_cloud, 2)
  net = tf_util.conv2d(net, 128, [1,1],
             padding='VALID', stride=[1,1],
             bn=True, is_training=is_training,
             scope='conv-reg1-stage1', bn_decay=bn_decay)
  net = tf_util.conv2d(net,  128, [1,1],
             padding='VALID', stride=[1,1],
             bn=True, is_training=is_training,
             scope='conv-reg2-stage1', bn_decay=bn_decay)
  net = tf_util.conv2d(net, 256, [1,1],
             padding='VALID', stride=[1,1],
             bn=True, is_training=is_training,
             scope='conv-reg3-stage1', bn_decay=bn_decay)
  net = tf_util.max_pool2d(net, [num_point,1],
    padding='VALID', scope='maxpool-stage1')
  net = tf.squeeze(net, axis=[1,2])
  net = tf.concat([net, predicted_classes_scores], axis=1)
  net = tf_util.fully_connected(net, 256, scope='fc1-stage1', bn=True,
    is_training=is_training, bn_decay=bn_decay)
  net = tf_util.fully_connected(net, 128, scope='fc2-stage1', bn=True,
    is_training=is_training, bn_decay=bn_decay)
  predicted_center = tf_util.fully_connected(net, 3, activation_fn=None,
    scope='fc3-stage1')
  # Translation
  stage1_center = predicted_center + mask_xyz_mean # Bx3

  # Get object point cloud in object coordinate
  object_point_cloud_xyz_new = \
    object_point_cloud - tf.expand_dims(predicted_center, 1)  
  
  # Amodal
  batch_size = object_point_cloud_xyz_new.get_shape()[0].value

  l0_xyz = object_point_cloud_xyz_new
  l0_points = None
  # Set abstraction layers
  l1_xyz, l1_points, l1_indices = pointnet_sa_module(l0_xyz, l0_points,
      npoint=128, radius=0.2, nsample=64, mlp=[64,64,128],
      mlp2=None, group_all=False,
      is_training=is_training, bn_decay=bn_decay, scope='ssg-layer1')
  l2_xyz, l2_points, l2_indices = pointnet_sa_module(l1_xyz, l1_points,
      npoint=32, radius=0.4, nsample=64, mlp=[128,128,256],
      mlp2=None, group_all=False,
      is_training=is_training, bn_decay=bn_decay, scope='ssg-layer2')
  l3_xyz, l3_points, l3_indices = pointnet_sa_module(l2_xyz, l2_points,
      npoint=None, radius=None, nsample=None, mlp=[256,256,512],
      mlp2=None, group_all=True,
      is_training=is_training, bn_decay=bn_decay, scope='ssg-layer3')

  # Fully connected layers
  net = tf.reshape(l3_points, [batch_size, -1])
  net = tf.concat([net, predicted_classes_scores], axis=1)
  net = tf_util.fully_connected(net, 512, bn=True,
      is_training=is_training, scope='fc1', bn_decay=bn_decay)
  net = tf_util.fully_connected(net, 256, bn=True,
      is_training=is_training, scope='fc2', bn_decay=bn_decay)

  # The first 3 numbers: box center coordinates (cx,cy,cz),
  # the next NUM_HEADING_BIN*2:  heading bin class scores and bin residuals
  # next NUM_SIZE_CLUSTER*4: box cluster scores and residuals
  output = tf_util.fully_connected(net,
      VAl_CONSTANT+VAL_NUM_HEADING_BIN*NUM_HEADING_BIN+VAL_NUM_SIZE_CLUSTER*NUM_SIZE_CLUSTER, activation_fn=None, scope='fc3')  
  #################################

  sess = tf.Session()
  # with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  pc = np.loadtxt(open("/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv", "rb"), delimiter=",")
#   print (pc)
  ops = {'pointclouds_pl': point_cloud,
           'is_training_pl': is_training}
#     feed_dict = {ops['pointclouds_pl']: [pc], ops['is_training_pl']: False}
  feed_dict = {ops['pointclouds_pl']: [pc]}
  batch_size_scores = sess.run(output, feed_dict=feed_dict)
  print(batch_size_scores.shape)
  print(batch_size_scores)
  # 		print(VAl_CONSTANT+VAL_NUM_HEADING_BIN*NUM_HEADING_BIN+VAL_NUM_SIZE_CLUSTER*NUM_SIZE_CLUSTER)

(1, 50)
[[ 4.170571   -0.9892492  -0.49729058 -0.2758702   2.3102226  -0.4403419
  -0.6460385  -0.7444961  -0.3599546   0.74409217 -0.9398809  -2.1423707
  -3.6984768   2.2100174   2.0242653  -2.612691   -4.0998425  -2.413827
  -0.8078402   0.6623677   0.09820342  0.9564769  -1.89658    -0.74844384
   2.0782728  -0.11387211 -3.9579349  -3.2002017  -2.4371717  -0.2291559
   1.1945076  -0.5025059  -2.1890934   2.1225324   2.5324304  -1.6708215
   2.39565    -3.1280456  -0.18804705  1.3580453   4.673561    3.8168316
  -0.22519392  1.3659784  -0.25010693  1.7068319  -3.5207624   0.3741892
  -0.30280694  1.5922778 ]]


In [0]:
# import random
# import csv

# with open('/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv', 'w', newline='') as writeFile:
# 	writer = csv.writer(writeFile)
# 	for x in range(1024):
# 		line = 	[0,0,0,0.0,0.0,0.0]
# 		line[0] = (random.randint(0,100))
# 		line[1] = (random.randint(0,100))
# 		line[2] = (random.randint(0,100))
# 		line[3] = (random.random())
# 		line[4] = (random.random())
# 		line[5] = (random.random())
# # 		print (line)
# 		writer.writerow(line)
# pc = np.loadtxt(open("/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/data_sample.csv", "rb"), delimiter=",")
# print (pc)

In [14]:
# Creates a graph.
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [24]:
!cat "/content/gdrive/My Drive/pointnet/PointnetEnhanced-master/pointnet2/utils/pointnet_util.py"

""" PointNet++ Layers

Author: Charles R. Qi
Date: November 2017
"""

import os
import sys
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
ROOT_DIR = os.path.dirname(BASE_DIR)
sys.path.append(os.path.join(ROOT_DIR, 'utils'))
sys.path.append(os.path.join(ROOT_DIR, 'tf_ops/sampling'))
sys.path.append(os.path.join(ROOT_DIR, 'tf_ops/grouping'))
sys.path.append(os.path.join(ROOT_DIR, 'tf_ops/3d_interpolation'))
from tf_sampling import farthest_point_sample, gather_point
from tf_grouping import query_ball_point, group_point, knn_point
from tf_interpolate import three_nn, three_interpolate
import tensorflow as tf
import numpy as np
import tf_util

def sample_and_group(npoint, radius, nsample, xyz, points, knn=False, use_xyz=True):
    '''
    Input:
        npoint: int32
        radius: float32
        nsample: int32
        xyz: (batch_size, ndataset, 3) TF tensor
        points: (batch_size, ndataset, channel) TF tensor, if None will just use xyz as points
        knn: bool, if True u

In [0]:
: